In [82]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
folder = "C:/Users/Vikrant/Desktop/Session/Case/"

In [83]:
NPPES_df = pd.read_csv(folder+"NPPES DATA_KOL.csv", sep = ",",quotechar = '"',\
                 dtype = object)
print (NPPES_df.head())
NPPES_df.dtypes

          NPI SPECIALTY   LAST_NAME FIRST_NAME MIDDLE_NAME  \
0  1003001363       OPH     STEVENS    CHARLES         NaN   
1  1003006115       RHM      DURHAM   BENJAMIN         NaN   
2  1003014986       OPM  YUZEFOVICH    MICHAEL         NaN   
3  1003017971       RHM       TWEET    MATTHEW         LEE   
4  1003029034      OPRS       BOWEN     JOSEPH    MARSHALL   

             ADDRESS_FIRST_LINE ADDRESS_SECOND_LINE        CITY STATE  \
0     1665 S IMPERIAL AVE STE D                 NaN   EL CENTRO    CA   
1      2300 MANCHESTER EXPY # A         SUITE 101-A    COLUMBUS    GA   
2  8302 OLD COURTHOUSE RD STE A                 NaN      VIENNA    VA   
3              2725 CAPITOL AVE           SUITE 302  SACRAMENTO    CA   
4            1296 E POLSTON AVE             SUITE B  POST FALLS    ID   

         ZIP  
0  922434247  
1  319046802  
2  221823873  
3  958166004  
4  838545217  


NPI                    object
SPECIALTY              object
LAST_NAME              object
FIRST_NAME             object
MIDDLE_NAME            object
ADDRESS_FIRST_LINE     object
ADDRESS_SECOND_LINE    object
CITY                   object
STATE                  object
ZIP                    object
dtype: object

In [84]:
KOL_df = pd.read_csv(folder+"30 KOLs.csv", sep = ",",quotechar = '"',\
                 dtype = object)
print (KOL_df.head())
KOL_df.count()

  LAST_NAME FIRST_NAME MIDDLE_NAME     ADDRESS_FIRST_LINE            CITY  \
0  SIDDaQUI     KHALID           A   951 N WASHINGTON AVE      TITUSVILLE   
1  ANDERSON    CHARBES       PETER         1400 S HWY 123      SAN MARCOS   
2      BABU      SARAT         NaN       51 W PROSPECT ST  EAST BRUNSWICK   
3      DAEY     SHARON           M  619 19TH STREET SOUTH      BIRMINGHAM   
4     PLOTZ       PHIL     BRADLEY         699 W TEFFT ST          NIPOMO   

  STATE        ZIP  
0    FL        NaN  
1    TX      78666  
2    NJ   88165711  
3    AL      35233  
4    CA  934449289  


LAST_NAME             30
FIRST_NAME            30
MIDDLE_NAME           24
ADDRESS_FIRST_LINE    30
CITY                  27
STATE                 30
ZIP                   26
dtype: int64

In [85]:
## As the column names are same select NPI and columns present in KOL list from NPPES data
NPPES_df =  NPPES_df.fillna('')
KOL_df =  KOL_df.fillna('')
NPPES_df['COMBINED_NPPES'] = ''
KOL_df['COMBINED'] = ''
for col in KOL_df.columns:
    if (col != 'COMBINED'):
        NPPES_df['COMBINED_NPPES'] = NPPES_df['COMBINED_NPPES']+ NPPES_df[col]
        KOL_df['COMBINED'] = KOL_df['COMBINED']+ KOL_df[col]
NPPES_df = NPPES_df[['COMBINED_NPPES','NPI']]
KOL_df.head()

,LAST_NAME,FIRST_NAME,MIDDLE_NAME,ADDRESS_FIRST_LINE,CITY,STATE,ZIP,COMBINED
0,SIDDaQUI,KHALID,A,951 N WASHINGTON AVE,TITUSVILLE,FL,,SIDDaQUIKHALIDA951 N WASHINGTON AVETITUSVILLEFL
1,ANDERSON,CHARBES,PETER,1400 S HWY 123,SAN MARCOS,TX,78666,ANDERSONCHARBESPETER1400 S HWY 123SAN MARCOSTX...
2,BABU,SARAT,,51 W PROSPECT ST,EAST BRUNSWICK,NJ,88165711,BABUSARAT51 W PROSPECT STEAST BRUNSWICKNJ88165711
3,DAEY,SHARON,M,619 19TH STREET SOUTH,BIRMINGHAM,AL,35233,DAEYSHARONM619 19TH STREET SOUTHBIRMINGHAMAL35233
4,PLOTZ,PHIL,BRADLEY,699 W TEFFT ST,NIPOMO,CA,934449289,PLOTZPHILBRADLEY699 W TEFFT STNIPOMOCA934449289


In [86]:
from fuzzywuzzy import process, fuzz
def fuzzy_match(x, choices, scorer, cutoff):
    return process.extractOne(
        x, choices=choices, scorer=scorer, score_cutoff=cutoff
    )

matching_results = KOL_df.loc[:, 'COMBINED'].apply(
    fuzzy_match,
    args=(
        NPPES_df.loc[:, 'COMBINED_NPPES'], 
        fuzz.ratio,
        80
    )
)
matching_results_df = matching_results.apply(pd.Series)
matching_results_df.columns=['COMBINED_NPPES','SCORE','INDEX']
matching_results_df['INDEX']

0     275
1     276
2     277
3     278
4     279
5     280
6     281
7     282
8     283
9     284
10    285
11    286
12    287
13    288
14    289
15    290
16    291
17    292
18    293
19    294
20    295
21    296
22    297
23    298
24    299
25    300
26    301
27    302
28    303
29    304
Name: INDEX, dtype: int64

In [87]:
KOL_NPI_df = NPPES_df.drop('COMBINED_NPPES',axis = 1).loc[matching_results_df['INDEX']]
KOL_NPI_df.to_csv(folder+'KOL_NPI.csv', index = False)